# Импортирую библиотеки

In [38]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import gpxpy
import os
from shapely.geometry import box, LineString
import matplotlib.pyplot as plt
import contextily as ctx
import time
import psycopg2
import cv2
import time
import math
import datetime
import concurrent.futures
import requests
from typing import List, Dict

# Создаю список с ссылками для скачивания

In [39]:
links = []

with open("./Links.txt", mode="r", encoding="UTF-8") as f:
    for i in f:
        links.append(i.strip())

In [40]:
links

['https://caucasia.ru/gpx/8EWQAV62.gpx',
 'https://caucasia.ru/gpx/CPOQQN9R.gpx',
 'https://caucasia.ru/gpx/CVBOS4W2.gpx',
 'https://caucasia.ru/gpx/HTGT4UOT.gpx',
 'https://caucasia.ru/gpx/HQUWT5UB.gpx',
 'https://caucasia.ru/gpx/S87VDJ9R.gpx',
 'https://caucasia.ru/gpx/XKSRMH5L.gpx',
 'https://caucasia.ru/gpx/S87VDJ9R.gpx',
 'https://caucasia.ru/gpx/G6I1K7J9.gpx']

# Загружаю gpx файлы из ссылок


In [41]:
for num, url in enumerate(links):
    try:
        response = requests.get(url)
        filename = f"track{num}.gpx"
        
        with open(f"data/gpx/{filename}", mode="wb") as f:
            f.write(response.content)
    except Exception as e:
        print("Ошибка при скачивании")


In [42]:
gpx_list = os.listdir("data/gpx")
gpx_list.sort()

In [43]:
gpx_list

['track0.gpx',
 'track1.gpx',
 'track2.gpx',
 'track3.gpx',
 'track4.gpx',
 'track5.gpx',
 'track6.gpx',
 'track7.gpx',
 'track8.gpx']

# Создаю изображения карты с маршрутом по трекам

In [44]:
margin = 0.02
result = []

for file in gpx_list:
    with open(f"{"data/gpx"}/{file}", mode="r", encoding="UTF-8") as f:
        gpx = gpxpy.parse(f)
    
    lats, lons, alts = [], [], []

    time_list = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                lats.append(point.latitude)
                lons.append(point.longitude)
                alts.append(point.elevation)
                result.append({
                        "track_id": file,
                        "analysis_date": point.time.date(),
                        "latitude": point.latitude,
                        "longitude": point.longitude,
                        "altitude": point.elevation})
                
    bbox = box(
            min(lons) - margin,
            min(lats) - margin,
            max(lons) + margin,
            max(lats) + margin
        )
    track_line = LineString(zip(lons, lats))

    gdf_bbox = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")
    gdf_bbox_web = gdf_bbox.to_crs(epsg=3857)

    gdf_track = gpd.GeoDataFrame(geometry=[track_line], crs="EPSG:4326")
    gdf_track_web = gdf_track.to_crs(epsg=3857)

    _, ax = plt.subplots(figsize=(10, 8))

    gdf_bbox_web.plot(ax=ax, alpha=0)
    gdf_track_web.plot(ax=ax, color="red", linewidth=2)

    ctx.add_basemap(ax, crs=gdf_bbox_web.crs, source=ctx.providers.OpenStreetMap.Mapnik)

    ax.set_axis_off()
    os.makedirs("data/image", exist_ok=True)

    plt.savefig(f"{"data/image"}/{file[:-4]}", dpi=150, bbox_inches="tight", pad_inches=0)
    plt.close()


In [45]:
ll = list(zip(lats, lons))

In [46]:
names_json = []

for lat, lon in (ll[0], ll[len(ll)//5],  ll[len(ll)//2], ll[len(ll)//3],  ll[-1]):
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            response = requests.get(f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json", headers=headers)
            print(response)
            json = response.json()
            time.sleep(1.5)

            names_json.append(json["address"]["county"])
        except Exception as e:
            print(f"Возникла ошибка при получении данных с API: {e}")

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


# Создаю базу данных

In [47]:
def connection():
     return psycopg2.connect(
        database="db_arthur", 
        user="arthur", 
        password="146a",
        host="localhost",
        port=5430)

In [48]:
try:
    connect = connection()
    
    cursor = connect.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS track_analysis (
            id SERIAL PRIMARY KEY,
            track_id VARCHAR,
            analysis_date TIMESTAMP,
            region VARCHAR,
            latitude DOUBLE PRECISION,
            longitude DOUBLE PRECISION,
            altitude DOUBLE PRECISION,
            step_frequency DOUBLE PRECISION,
            temperature DOUBLE PRECISION,
            terrain_type VARCHAR,
            key_objects VARCHAR,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    for i in result:
        cursor.execute("""
            INSERT INTO track_analysis
            (track_id, analysis_date, latitude, longitude, altitude)
            VALUES (%s, %s, %s, %s, %s)
        """,
            (i["track_id"],
            i["analysis_date"],
            i["latitude"],
            i["longitude"],
            i["altitude"]))

    connect.commit()
except Exception as e:
    print(f"Ошибка при создании базы данных: {e}")
finally:
    if connect:
        cursor.close()
        connect.close()

создаю датасет и преобразую колонку analysis_date в удобный формат даты

In [67]:
df_test = pd.DataFrame(result)
df_test['analysis_date'] = pd.to_datetime(df_test['analysis_date']).dt.strftime('%Y-%m-%d')

In [50]:
df_test

,track_id,analysis_date,latitude,longitude,altitude
0,track0.gpx,2025-11-17,44.589199,38.400221,187.926320
1,track0.gpx,2025-11-17,44.589160,38.400190,188.159584
2,track0.gpx,2025-11-17,44.589230,38.400040,188.616480
3,track0.gpx,2025-11-17,44.589560,38.399690,189.506496
4,track0.gpx,2025-11-17,44.590050,38.399120,188.224640
...,...,...,...,...,...
32001,track8.gpx,2006-11-26,44.736280,37.772490,0.610000
32002,track8.gpx,2006-11-26,44.736330,37.771890,43.920000
32003,track8.gpx,2006-11-26,44.736140,37.773240,5.420000
32004,track8.gpx,2006-11-26,44.736260,37.771970,5.420000


# Получаю температуру

In [51]:
def temp(lat, lon, date):
    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        'latitude': lat,        # Ширина
        'longitude': lon,       # Долгота
        'start_date': date,
        'end_date': date,
        'hourly': 'temperature_2m',
        "timezone":"auto"
    }
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0'}

    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    if response.status_code == 200:
        data = response.json()
        return data["hourly"]['temperature_2m'][12]
    else:
        print(f"Данные не получены")

In [52]:
temp(44.539757, 33.882218, "2021-01-05")

12.2

In [53]:
from geopy.distance import geodesic

def distance(lat1, lon1, lat2, lon2):
    """
    Рассчитывает расстояние в метрах между двумя точками
    """
    
    point1 = (lat1, lon1)
    point2 = (lat2, lon2)
    
    distance = geodesic(point1, point2).meters
    return round(distance / 0.75, 3)

In [54]:
def analysis_weather(df_test):
    '''
    Функция для интерполяции температур между опорными точками.
    Использует 5 опорных точек для интерполяции температуры для всех строк.
    '''
    
    n = len(df_test)
    key_indexes = [0, n//4, n//2, 3*n//4, n-1]
    temperatures_at_key_points = {}
    
    for idx in key_indexes:
        lat = df_test.iloc[idx]["latitude"]
        lon = df_test.iloc[idx]["longitude"]
        date = df_test.iloc[idx]["analysis_date"]
        
        temp_value = temp(lat, lon, date)
        
        temperatures_at_key_points[idx] = temp_value

    
    if len(temperatures_at_key_points) < 2:
        print("Недостаточно данных для интерполяции, проверьте работоспособность API")
        df_test["temperature"] = None
        return df_test
    
    all_temperatures = []
    all_steps = []
    left_idx = 0
    right_idx = 1
    
    for i in range(n):
        # Если текущий индекс превысил правую границу и есть следующий интервал
        if (right_idx < len(key_indexes) - 1 and i >= key_indexes[right_idx]):
            left_idx += 1
            right_idx += 1
        
        left_key = key_indexes[left_idx]
        right_key = key_indexes[right_idx]
        
        left_temp = temperatures_at_key_points[left_key]
        right_temp = temperatures_at_key_points[right_key]
        
        # Если текущая строка - одна из опорных точек
        if i in temperatures_at_key_points:
            temperature = temperatures_at_key_points[i]
        elif left_temp is not None and right_temp is not None:
            # Линейная интерполяция между двумя опорными точками
            temperature = left_temp + (right_temp - left_temp) * (i - left_key) / (right_key - left_key)
        else:
            # Если нет данных для интерполяции
            temperature = left_temp if left_temp is not None else right_temp
        
        all_temperatures.append(temperature)
    
    df_test = df_test.copy()
    df_test["temperature"] = all_temperatures
    # df_test["step_frequency"] = all_steps
    
    return df_test

In [55]:
try:
    df_test = analysis_weather(df_test)
except Exception as e:
    print(f"Ошибка вызова функции: analysis_weather {e}")


добоавляю в базу данных температуру

In [56]:
try:
    connect_db = connection()
    cursor = connect_db.cursor()

    for idx, row in df_test.iterrows():
        cursor.execute("""
            UPDATE track_analysis 
            SET temperature = %s 
            WHERE id = %s
        """,
        (row["temperature"], idx + 1))  # +1 если id начинается с 1

    connect_db.commit()
    print(f"Обновлено {len(df_test)} записей в таблице track_analysis")

except Exception as e:
    print(f"Ошибка при обновлении базы данных: {e}")
finally:
    if connect_db:
        cursor.close()
        connect_db.close()

Обновлено 32006 записей в таблице track_analysis


In [57]:
def extract_map_region(lat: float, lon: float):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json", headers=headers)
        json = response.json()
        time.sleep(1.5)
        if "county" in json["address"]:
            return json["address"]["county"]
        if "state" in json["address"]:
            return json["address"]["state"]
        return json["address"]["country"]
    except Exception as e:
        print(f"Error {e}")

In [58]:
n = len(df_test)
key_indexes = (0, n//4, n//2, 3*n//4, -1)


regions = []
for idx in key_indexes:
    region = extract_map_region(df_test.iloc[idx]['latitude'], df_test.iloc[idx]['longitude'])
    regions.append(region)

In [70]:
df_test

,track_id,analysis_date,latitude,longitude,altitude
0,track0.gpx,2025-11-17,44.589199,38.400221,187.926320
1,track0.gpx,2025-11-17,44.589160,38.400190,188.159584
2,track0.gpx,2025-11-17,44.589230,38.400040,188.616480
3,track0.gpx,2025-11-17,44.589560,38.399690,189.506496
4,track0.gpx,2025-11-17,44.590050,38.399120,188.224640
...,...,...,...,...,...
32001,track8.gpx,2006-11-26,44.736280,37.772490,0.610000
32002,track8.gpx,2006-11-26,44.736330,37.771890,43.920000
32003,track8.gpx,2006-11-26,44.736140,37.773240,5.420000
32004,track8.gpx,2006-11-26,44.736260,37.771970,5.420000


In [86]:
x1, y1 = 0, 0
x2, y2 = 0, 0


for i in df_test.index[0:1]:
    x1 = df_test["latitude"][i]
    y1 = df_test["longitude"][i]
for j in df_test.index[1:2]:
    x2 = df_test["latitude"][j]
    y2 = df_test["longitude"][j]

print(x1, y1)
print(x2, y2)
    

44.58919885044738 38.40022087097168
44.58916 38.40019


In [ ]:
def step_frequency():
    """
    нужно написать функцию которая идет по трекам и id, и вычисляет сколько прошло времени между 1 и 2 строкой, и записывает это значение в новую колонку
    """


    for i in df_test["track_id"]:
        

In [59]:
df_test

,track_id,analysis_date,latitude,longitude,altitude,temperature
0,track0.gpx,2025-11-17,44.589199,38.400221,187.926320,14.400000
1,track0.gpx,2025-11-17,44.589160,38.400190,188.159584,14.399163
2,track0.gpx,2025-11-17,44.589230,38.400040,188.616480,14.398325
3,track0.gpx,2025-11-17,44.589560,38.399690,189.506496,14.397488
4,track0.gpx,2025-11-17,44.590050,38.399120,188.224640,14.396650
...,...,...,...,...,...,...
32001,track8.gpx,2006-11-26,44.736280,37.772490,0.610000,12.086702
32002,track8.gpx,2006-11-26,44.736330,37.771890,43.920000,12.090026
32003,track8.gpx,2006-11-26,44.736140,37.773240,5.420000,12.093351
32004,track8.gpx,2006-11-26,44.736260,37.771970,5.420000,12.096675


In [60]:
distance(43.97778977639973, 39.85064433887601, 43.97778240032494, 39.850631095469)

1.789